Logistic Regression with PCA 

private - 0.661, public - 0.694

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler


train_url = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/train.csv'
test_url = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/test.csv'

In [ ]:
train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

targets = train['target']
train = train.drop(columns=['id', 'target'])

ids = test['id']
test = test.drop(columns=['id'])


transformer = RobustScaler().fit(train.values)
trans_x = transformer.transform(train.values)
trans_test = transformer.transform(test.values)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
scores = []
param_grid = {"C": [.01, .1, 1, 10, 100, 1000],
                "penalty": ('l1','l2')}
clf = LogisticRegression(random_state=0, solver='liblinear', class_weight='balanced', max_iter = 1000, penalty='l1')
gs = GridSearchCV(clf, param_grid, cv=5)
gs.fit(trans_x, targets)

best_score = gs.best_score_
best_estimator = gs.best_estimator_
best_pca = 'N/a'
print(best_score)
"""
for i in range(10,250, 10):
    pca = PCA(n_components=i)
    pca.fit(train.values)
    
    X = pca.transform(trans_x)
    transformed_test = pca.transform(test.values)
    #Find the best model for these components
    param_grid = {"C": [.01, .1, 1, 10, 100, 1000],
                "penalty": ('l1','l2')}
    clf = LogisticRegression(random_state=0, solver='liblinear', class_weight='balanced', max_iter = 1000)
    gs = GridSearchCV(clf, param_grid, cv=5)
    gs.fit(X, targets)
    print("Best Score for : ", str(i), " is ", str(gs.best_score_))
    scores.append(gs.best_score_)
    if gs.best_score_ > best_score:
        best_score = gs.best_score_
        best_estimator = gs.best_estimator_
        best_pca = i
print("Best score was : ", str(best_score), " with estimator ", str(best_estimator), ' \n and PCA : ', str(best_pca))
"""
pca = PCA(n_components=2)
pca.fit(train.values)
vals_2d = pca.transform(train.values)
import matplotlib.pyplot as plt
colors = ['red' if t == 1 else 'blue' for t in targets]
for i in range(len(vals_2d)):
    plt.scatter(x=vals_2d[i,0], y=vals_2d[i,1], color=colors[i])

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_validate
selector = RFECV(best_estimator, step=1, cv=5)
selector = selector.fit(trans_x, targets)
new_trans_x = selector.transform(trans_x)
new_trans_test = selector.transform(trans_test)

In [ ]:
param_grid = {"C": [.01, .1],
                "penalty": ('l1','l2')}
clf = LogisticRegression(random_state=0, solver='liblinear', class_weight='balanced', max_iter = 1000, penalty='l1')
gs = GridSearchCV(clf, param_grid, cv=5)
gs.fit(new_trans_x, targets)
print(gs.best_estimator_)
print(gs.best_score_)
print(gs.score(new_trans_x, targets))
predictions = gs.predict_proba(new_trans_test)[:,1]

In [ ]:
pred_result_list = [int(round(x)) for x in predictions]

pred_result_list

In [ ]:
filePath = '../result_folder/logistic_V8.csv'

In [ ]:
import csv

with open(filePath, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "target"])
    for i in range(250, 20000):
        writer.writerow([i, pred_result_list[i-250]])
    file.close()